<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/Image/get_image_id.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/Image/get_image_id.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=Image/get_image_id.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/Image/get_image_id.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell.

In [1]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [2]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for this first time or if you are getting an authentication error.  

In [3]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [4]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [5]:
# This function masks the input with a threshold on the simple cloud score.


def cloudMask(img):
    cloudscore = ee.Algorithms.Landsat.simpleCloudScore(img).select('cloud')
    return img.updateMask(cloudscore.lt(50))


# Load a Landsat 5 image collection.
collection = ee.ImageCollection('LANDSAT/LT5_L1T_TOA') \
    .filterDate('2008-04-01', '2010-04-01') \
    .filterBounds(ee.Geometry.Point(-122.2627, 37.8735)) \
    .map(cloudMask) \
    .select(['B4', 'B3']) \
    .sort('system:time_start', True)  # Sort the collection in chronological order.

print(collection.size().getInfo())

first = collection.first()
propertyNames = first.propertyNames()
print(propertyNames.getInfo())

uid = first.get('system:id')
print(uid.getInfo())


38
['system:version', 'system:id', 'RADIANCE_MULT_BAND_5', 'RADIANCE_MULT_BAND_6', 'IMAGE_QUALITY', 'RADIANCE_MULT_BAND_3', 'RADIANCE_MULT_BAND_4', 'RADIANCE_MULT_BAND_1', 'RADIANCE_MULT_BAND_2', 'google:synthesized_metadata', 'CORRECTION_GAIN_BAND_2', 'CORRECTION_GAIN_BAND_1', 'system:footprint', 'REFLECTIVE_SAMPLES', 'CORRECTION_GAIN_BAND_7', 'CORRECTION_GAIN_BAND_6', 'CORRECTION_GAIN_BAND_5', 'SUN_AZIMUTH', 'CORNER_UL_PROJECTION_Y_PRODUCT', 'CORRECTION_GAIN_BAND_4', 'CORRECTION_GAIN_BAND_3', 'CPF_NAME', 'CORNER_LR_PROJECTION_Y_PRODUCT', 'DATE_ACQUIRED', 'ELLIPSOID', 'google:registration_offset_x', 'google:registration_offset_y', 'SENSOR_MODE', 'STATION_ID', 'RESAMPLING_OPTION', 'ORIENTATION', 'WRS_ROW', 'RADIANCE_MULT_BAND_7', 'QUANTIZE_CAL_MIN_BAND_7', 'QUANTIZE_CAL_MIN_BAND_6', 'QUANTIZE_CAL_MIN_BAND_5', 'CLOUD_COVER', 'QUANTIZE_CAL_MIN_BAND_4', 'QUANTIZE_CAL_MIN_BAND_3', 'QUANTIZE_CAL_MIN_BAND_2', 'QUANTIZE_CAL_MIN_BAND_1', 'CORNER_LL_LON_PRODUCT', 'CORNER_LR_LON_PRODUCT', 'GRID_

## Display Earth Engine data layers 

In [6]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map